In [ ]:
using WAV;
using Gen;
import Random;
include("model.jl");
include("plotting.jl");
include("proposals.jl");
include("gammatonegram.jl");
include("time_helpers.jl");
include("inference_helpers.jl");
include("initialization.jl");

In [ ]:
println("Loading in demo name...")

demo_name = "Track32slow_new"
demo_names = ["01-df2_dt125-shorttone-smalldf","01-Track15higherBetweens","02-df2_dt150-shorttone-smalldf","02-Track15lowerBetweens","03-df2_dt175-shorttone-smalldf","04-df2_dt200-shorttone-smalldf","05-df4_dt125-shorttone-smalldf","06-df4_dt150-shorttone-smalldf","07-df4_dt175-shorttone-smalldf","08-df4_dt200-shorttone-smalldf","09-df6_dt125-shorttone-smalldf","10-df6_dt150-shorttone-smalldf","11-df6_dt175-shorttone-smalldf","12-df6_dt200-shorttone-smalldf","13-df8_dt125-shorttone-smalldf","14-df8_dt150-shorttone-smalldf","15-df8_dt175-shorttone-smalldf","16-df8_dt200-shorttone-smalldf","1iii","1ii","1iv","1i","1v","2iii","2ii","2iv","2i","2vi","2v","4ii","continuity","Track17X","Track32fast_new","Track32slow_new"]
for i = [27]
    demo_name = demo_names[i]
    demo_gram, wts, scene_duration, audio_sr = load_sound(demo_name)
    plot_gtg(demo_gram, scene_duration, audio_sr/2.); 
end


In [ ]:
max_elements=10; duration=1.0; audio_sr=20000; steps = Dict(:t=>0.020, :min=> 0.020, :f=>4); annealing_noise=-1
wts, f = gtg_weights(audio_sr)

# ## Inference parameters
# annealing_noise = 1.5
# n_steps = 1
# if annealing_noise > 0 
#     slope = 0.5; midpoint = 200; height = 0.7; base = 0.8; cutoff = 300
#     annealing_schedule = make_annealing_schedule(slope, midpoint, height, base, cutoff)
# end

In [ ]:
println("Loading in sound...")
## Inference
#Load in sound observation and its neural network guide distribution
demo_gram, wts, scene_duration, audio_sr = load_sound(demo_name)
demo_guide = read_guide(demo_name)

In [ ]:
plot_gtg(demo_gram, scene_duration, audio_sr/2.);

In [ ]:
println("Doing inference...")
#Sample initialization from the guide distribution 
args = (max_elements, float(scene_duration), wts, steps, Int(audio_sr), annealing_noise)
n_likelihood_samples = 1
data_init_trace = init_from_likelihood(n_likelihood_samples, demo_guide, demo_gram, args); 
plot_sources(data_init_trace, demo_gram, 0)

In [ ]:
n_steps=2
traces, accept_counts, proposal_counts = run_inference(mcmc_update, data_init_trace, demo_gram, n_steps);
println("Inference complete!")

In [ ]:
println("Saving our results.")
## Save results
PyPlot.ioff()
# Show frequency of proposal acceptance for each proposal type
plot_proposals(accept_counts, proposal_counts, annealing_noise)


In [ ]:
for i = 1:length(traces)
    plot_sources(traces[i], demo_gram, i)    
end

In [ ]:
println(get_choices(traces[end]))